In [1]:
import numpy as np
import pandas as pd

In [8]:
# Reading the CSV file here (Actual Dataset that we are cleaning)

RAW_PATH = "Building_Permits.csv"
FINAL_PATH = "Final_Clean.csv"

pd.set_option("display.max_columns", None)
# Loading Data Here
df = pd.read_csv(RAW_PATH, low_memory=False)

In [ ]:
def title_case(df, cols):
    df[cols] = df[cols].apply(lambda c: c.str.title())
    return df

In [ ]:

def fill_from_other(df, target, source):
    df[target] = df[target].fillna(df[source])
    return df

In [ ]:

def standardize_street_suffix(series):
    mapping = {
        'Av': 'Av', 'Avenue': 'Av',
        'St': 'St', 'Street': 'St',
        'Bl': 'Blvd', 'Boulevard': 'Blvd',
        'Dr': 'Dr', 'Drive': 'Dr',
        'Tr': 'Trl', 'Trail': 'Trl'
    }
    return series.replace(mapping)


In [ ]:
permit_map = {
    1: 'One', 2: 'Two', 3: 'Three', 4: 'Four',
    5: 'Five', 6: 'Six', 7: 'Seven', 8: 'Eight'
}

df['Permit Type'] = df['Permit Type'].replace(permit_map)


In [ ]:
text_cols = [
    'Permit Type Definition', 'Current Status',
    'Description', 'Existing Use', 'Proposed Use'
]
df = title_case(df, text_cols)

In [ ]:
df['Lot'] = (
    df['Lot']
    .astype(str)
    .str.replace(r'[A-Za-z]', '', regex=True)
    .astype(float)
)


In [ ]:
df['Street Suffix'] = standardize_street_suffix(df['Street Suffix'])

In [ ]:
drop_cols = [
    'Structural Notification', 'TIDF Compliance',
    'Site Permit', 'Fire Only Permit',
    'Completed Date', 'Voluntary Soft-Story Retrofit',
    'Unit', 'Unit Suffix'
]

df.drop(columns=drop_cols, inplace=True)

In [ ]:
df = fill_from_other(df, 'Proposed Use', 'Existing Use')
df = fill_from_other(df, 'Estimated Cost', 'Revised Cost')
df = fill_from_other(df, 'Existing Use', 'Proposed Use')
df = fill_from_other(df, 'Existing Units', 'Proposed Units')


In [ ]:
required_cols = [
    'Supervisor District', 'Zipcode',
    'Neighborhoods - Analysis Boundaries',
    'Estimated Cost', 'Revised Cost',
    'Plansets', 'Existing Construction Type',
    'Proposed Construction Type'
]

df.dropna(subset=required_cols, inplace=True)

In [ ]:
df['Issued Date'].fillna('Not Approved Yet', inplace=True)
df['Permit Expiration Date'].fillna('Not Finalized Yet', inplace=True)
df['First Construction Document Date'].fillna('Not Started Yet', inplace=True)


In [ ]:
int_cols = [
    'Zipcode', 'Plansets',
    'Existing Units', 'Proposed Units',
    'Supervisor District',
    'Estimated Cost', 'Revised Cost',
    'Number of Existing Stories',
    'Number of Proposed Stories'
]

df[int_cols] = df[int_cols].astype(int)


In [ ]:
df.rename(
    columns={'Existing Construction Type Description':
             'Existing/Proposed Construction Type Description'},
    inplace=True
)

df.drop(columns=['Proposed Construction Type Description'], errors='ignore', inplace=True)

In [ ]:
null_ratio = df.isnull().mean()
assert null_ratio.max() < 0.01, "Unexpected nulls detected"

In [7]:
df.to_csv(FINAL_PATH, index=False)
print(f"Final Shape: {df.shape}")

Final Shape: (198900, 43)
